# Data Cleaning EV sales 

In [ ]:
# importing neccessry libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action = "ignore")

In [55]:
# loading the dataset
data_his = pd.read_csv("data/historical_data.csv")
data_pro = pd.read_csv("data/data_project-APS.csv")



In [56]:
# check data description
data_his.info()
data_pro.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      3798 non-null   object 
 1   category    3798 non-null   object 
 2   parameter   3798 non-null   object 
 3   mode        3798 non-null   object 
 4   powertrain  3798 non-null   object 
 5   year        3798 non-null   int64  
 6   unit        3798 non-null   object 
 7   value       3798 non-null   float64
dtypes: float64(1), int64(1), object(6)
memory usage: 237.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      457 non-null    object 
 1   category    457 non-null    object 
 2   parameter   457 non-null    object 
 3   mode        457 non-null    object 
 4   powertrain  457 non-null    object 
 5   year        457 non-null

In [57]:
# check first 10 rows 
data_his.head(10)

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.00000
1,Australia,Historical,EV stock share,Cars,EV,2011,percent,0.00039
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.00650
3,Australia,Historical,EV stock,Cars,BEV,2011,Vehicles,49.00000
4,Australia,Historical,EV stock,Cars,BEV,2012,Vehicles,220.00000
5,Australia,Historical,EV stock,Cars,PHEV,2012,Vehicles,80.00000
6,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80.00000
7,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.03000
8,Australia,Historical,EV stock share,Cars,EV,2012,percent,0.00240
9,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.00000


In [58]:
#check for duplicates
print(data_his.duplicated().sum())
print(data_pro.duplicated().sum())

0
0


In [59]:
# Get unique powertrain types
unique_powertrains = data_his['powertrain'].unique()
print(unique_powertrains)


['BEV' 'EV' 'PHEV' 'FCEV']


In [60]:
# Filter rows that are historical ev sales and sales share data 
filtered_data = data_his[(data_his['parameter'] == 'EV sales') | (data_his['parameter'] == 'EV sales share')]
filtered_data.head()


,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,Vehicles,49.0000
2,Australia,Historical,EV sales share,Cars,EV,2011,percent,0.0065
6,Australia,Historical,EV sales,Cars,PHEV,2012,Vehicles,80.0000
7,Australia,Historical,EV sales share,Cars,EV,2012,percent,0.0300
9,Australia,Historical,EV sales,Cars,BEV,2012,Vehicles,170.0000


In [61]:
# Create a ev_sales historical table with selected columns
df_ev_sales_his = filtered_data[['region', 'year', 'powertrain','unit','value']].reset_index(drop=True)
df_ev_sales_his.head()

,region,year,powertrain,unit,value
0,Australia,2011,BEV,Vehicles,49.0000
1,Australia,2011,EV,percent,0.0065
2,Australia,2012,PHEV,Vehicles,80.0000
3,Australia,2012,EV,percent,0.0300
4,Australia,2012,BEV,Vehicles,170.0000


In [62]:
# Seperate sales volume and sales share percentage
# sales volume table for each power type
sales_volumes =df_ev_sales_his[df_ev_sales_his['unit'] == 'Vehicles'][['region', 'year', 'powertrain', 'value']]
sales_volumes.rename(columns={'value': 'sales_volumes'}, inplace=True)
sales_volumes.head()

# sales volumn total for all EV cars sales
sales_volumes_tt=sales_volumes.groupby(['region', 'year']).agg({'sales_volumes': 'sum'}).reset_index()
sales_volumes_tt.head()

# sales shares table for all EV cars
sales_shares = df_ev_sales_his[df_ev_sales_his['unit'] == 'percent'][['region', 'year', 'powertrain', 'value']]
sales_shares.rename(columns={'value': 'sales_shares'}, inplace=True)

# Merge the two DataFrames
clean_ev_sales_data_his= pd.merge(sales_shares, sales_volumes_tt, on=['region', 'year'], how='outer')

# Display the clean DataFrame
clean_ev_sales_data_his.head()


,region,year,powertrain,sales_shares,sales_volumes
0,Australia,2011,EV,0.0065,49.0
1,Australia,2012,EV,0.0300,250.0
2,Australia,2013,EV,0.0340,290.0
3,Australia,2014,EV,0.1600,1320.0
4,Australia,2015,EV,0.2000,1760.0


*cleaning the sales projection data*

In [63]:
# Filter rows where project ev sales and sales share data and year greater than 2024
filtered_data_pro = data_pro[
    ((data_pro['parameter'] == 'EV sales') | (data_pro['parameter'] == 'EV sales share')) & 
    (data_pro['year'] > 2024)
].reset_index(drop=True)

filtered_data_pro.head()


,region,category,parameter,mode,powertrain,year,unit,value
0,China,Projection-APS,EV sales,Cars,BEV,2025,Vehicles,8300000.0
1,China,Projection-APS,EV sales,Cars,FCEV,2025,Vehicles,27000.0
2,China,Projection-APS,EV sales,Cars,PHEV,2025,Vehicles,2600000.0
3,China,Projection-APS,EV sales share,Cars,EV,2025,percent,49.0
4,China,Projection-APS,EV sales share,Cars,EV,2030,percent,68.0


In [64]:
# Create a ev_sales projection table with selected columns
df_ev_sales_pro = filtered_data_pro[['region', 'year', 'powertrain', 'unit', 'value']]
print(df_ev_sales_pro['region'].unique())
df_ev_sales_pro.head()


['China' 'Europe' 'India' 'Rest of the world' 'USA' 'World']


,region,year,powertrain,unit,value
0,China,2025,BEV,Vehicles,8300000.0
1,China,2025,FCEV,Vehicles,27000.0
2,China,2025,PHEV,Vehicles,2600000.0
3,China,2025,EV,percent,49.0
4,China,2030,EV,percent,68.0


In [65]:
# Seperate sales volume and sales share percentage
# sales volume table for each power type
sales_volumes_pro =df_ev_sales_pro[df_ev_sales_pro['unit'] == 'Vehicles'][['region', 'year', 'powertrain', 'value']]
sales_volumes_pro.rename(columns={'value': 'sales_volumes'}, inplace=True)
sales_volumes_pro.head()

# sales volumn total for all EV cars sales
sales_volumes_tt_pro=sales_volumes_pro.groupby(['region', 'year']).agg({'sales_volumes': 'sum'}).reset_index()
sales_volumes_tt_pro.head()

# sales shares table for all EV cars
sales_shares_pro = df_ev_sales_pro[df_ev_sales_pro['unit'] == 'percent'][['region', 'year', 'powertrain', 'value']]
sales_shares_pro.rename(columns={'value': 'sales_shares'}, inplace=True)

# Merge the two DataFrames
clean_ev_sales_data_pro= pd.merge(sales_shares_pro, sales_volumes_tt_pro, on=['region', 'year'], how='outer')

# Display the clean DataFrame
clean_ev_sales_data_pro.head()


,region,year,powertrain,sales_shares,sales_volumes
0,China,2025,EV,49.0,10927000.0
1,China,2030,EV,68.0,17610000.0
2,China,2035,EV,85.0,22160000.0
3,Europe,2025,EV,31.0,4705800.0
4,Europe,2030,EV,62.0,9310000.0


****Cleaning EV charging points data****

In [66]:
# loading the dataset
chargingp_his = pd.read_csv("data/charging points_his.csv")
chargingp_pro = pd.read_csv("data/charging points_pro.csv")

In [67]:
# check data description
chargingp_his.info()
chargingp_pro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   region      750 non-null    object 
 1   category    750 non-null    object 
 2   parameter   750 non-null    object 
 3   mode        750 non-null    object 
 4   powertrain  750 non-null    object 
 5   year        750 non-null    int64  
 6   unit        750 non-null    object 
 7   value       750 non-null    float64
dtypes: float64(1), int64(1), object(6)
memory usage: 47.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   region      84 non-null     object
 1   category    84 non-null     object
 2   parameter   84 non-null     object
 3   mode        84 non-null     object
 4   powertrain  84 non-null     object
 5   year        84 non-null     int64 
 

In [68]:
#check for duplicates
print(chargingp_his.duplicated().sum())
print(chargingp_pro.duplicated().sum())

0
0


In [69]:
# read the frist 10 rows
chargingp_his.head(10)

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
1,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
2,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
3,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
4,Australia,Historical,EV charging points,EV,Publicly available slow,2019,charging points,1700.0
5,Australia,Historical,EV charging points,EV,Publicly available slow,2020,charging points,2300.0
6,Australia,Historical,EV charging points,EV,Publicly available fast,2021,charging points,320.0
7,Australia,Historical,EV charging points,EV,Publicly available slow,2021,charging points,2000.0
8,Australia,Historical,EV charging points,EV,Publicly available fast,2022,charging points,470.0
9,Australia,Historical,EV charging points,EV,Publicly available slow,2022,charging points,2100.0


In [70]:
# group by region and year, then sum the charging points for all types (fast and slow) 
df_chargingp_his =chargingp_his.groupby(['region', 'year']).agg({'value': 'sum'}).rename(columns={'value':'charging_points'}).reset_index()
df_chargingp_his.head()

,region,year,charging_points
0,Australia,2017,480.0
1,Australia,2018,731.0
2,Australia,2019,1700.0
3,Australia,2020,2300.0
4,Australia,2021,2320.0


In [71]:
# read the frist 10 rows
chargingp_pro.head(10)

,region,category,parameter,mode,powertrain,year,unit,value
0,China,Projection-APS,EV charging points,EV,Publicly available fast,2020,charging points,310000
1,China,Projection-APS,EV charging points,EV,Publicly available slow,2020,charging points,500000
2,China,Projection-APS,EV charging points,EV,Publicly available fast,2021,charging points,470000
3,China,Projection-APS,EV charging points,EV,Publicly available slow,2021,charging points,680000
4,China,Projection-APS,EV charging points,EV,Publicly available fast,2022,charging points,760000
5,China,Projection-APS,EV charging points,EV,Publicly available slow,2022,charging points,1000000
6,China,Projection-APS,EV charging points,EV,Publicly available fast,2023,charging points,1200000
7,China,Projection-APS,EV charging points,EV,Publicly available slow,2023,charging points,1500000
8,China,Projection-APS,EV charging points,EV,Publicly available fast,2025,charging points,2100000
9,China,Projection-APS,EV charging points,EV,Publicly available slow,2025,charging points,2600000


In [72]:
# filter data with year great than 2024 and rename the value to station unit
df_filtered = chargingp_pro[chargingp_pro['year']>2024]
# group by region and year, then sum the values for all types (fast and slow)
df_chargingp_pro=df_filtered.groupby(['region', 'year']).agg({'value': 'sum'}).rename(columns={'value':'charging_points'}).reset_index()
df_chargingp_pro.head()


,region,year,charging_points
0,China,2025,4700000
1,China,2030,10800000
2,China,2035,15500000
3,Europe,2025,1060000
4,Europe,2030,1750000


****Save the cleaned datasets****

In [74]:
#Save the result to a CSV file
sales_volumes.to_csv('cleaned data/sales_vol_his_bytype.csv', index=False)
clean_ev_sales_data_his.to_csv('cleaned data/ev_sales_his.csv', index=False)
sales_volumes_pro.to_csv('cleaned data/sales_vol_pro_bytype.csv', index=False)
clean_ev_sales_data_pro.to_csv('cleaned data/ev_sales_pro.csv', index=False)
df_chargingp_his.to_csv('cleaned data/ev_chargingpoint_his.csv', index=False)
df_chargingp_pro.to_csv('cleaned data/ev_chargingpoint_pro.csv', index=False)